In [1]:
%matplotlib inline

In [103]:
import pandas as pd
from bqplot import *
import us
import bqplot.pyplot as plt
from ipywidgets import (widgets, HBox, VBox, Layout, interact)

In [3]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]

fn = 'ufo-scrubbed-geocoded-time-standardized.csv'
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
ufo_date = ufo.set_index("date")
ufo['year'] = ufo_date.index.year
ufo = ufo[ufo['fips'] != -1]

In [4]:
fips_count = ufo.groupby(['fips','year'])['duration_seconds'].count()
fips_sum = ufo.groupby(['fips','year'])['duration_seconds'].sum()
fips = pd.DataFrame()

In [5]:
fips['total_sightings'] = fips_count
fips['duration_seconds'] =fips_sum

In [6]:
fips = fips.reset_index(level=['fips','year'])

In [7]:
## get population of each state
population=pd.read_csv('U.S._states_population.csv',na_values='n/a')
population=population[(population['year']>=1910)&(population['year']<=2014)]
ppl=pd.melt(population, id_vars=['year'], value_vars=list(population)[1:])
ppl=ppl.rename(index=str, columns={"variable": "state", "value": "population"})
ppl["fips"] = ppl["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
ppl['population']=ppl['population'].str.replace(",", "").astype(float)

In [8]:
## Merge population data with fips and normalize data
fips=pd.merge(fips, ppl, how='left', on=['fips', 'year'])
fips['normalized_total_sightings']=fips.total_sightings/fips.population*1000000
fips['normalized_duration_seconds']=fips.duration_seconds/fips.population*1000000

In [9]:
import widgetsnbextension
from IPython.display import display

In [104]:
@interact(field = ["total_sightings", "duration_seconds"], #create drop box of field
          normalized = ["Yes","No"], #normalized by population
          year = (min(fips['year']),max(fips['year']),1))#create time slider

def make_plot(field="total_sightings", year=max(fips['year']), normalized="No"):
    fips_year = fips[fips['year']==int(year)]
    if normalized == 'Yes':
        field='normalized_'+field
    def_tt = Tooltip(fields=['name', 'color'], formats=['', ''], labels=['State', field])
    map_res = Map(map_data=topo_load('map_data/USStatesMap.json'),
                      colors = {'default_color':'Black'},
                      scales = {'projection': AlbersUSA(),
                                'color': ColorScale(colors=['DeepSkyBlue','Gray','Red'])},
                      color=dict(zip(fips_year['fips'],fips_year[field])),
                      tooltip=def_tt) #refered to starting code from Prof. Turk
    map_fig = Figure(marks=[map_res], title="Visualizing UFO")
    
    map_res.interactions={'click':'tooltip', 'hover':'tooltip'}
    
    dt_x = LinearScale()
    sc_y = LinearScale()
    ax_x = Axis(scale=dt_x)
    ax_y = Axis(scale=sc_y, orientation='vertical', label='')
    line_marks = Lines(x=[], y=[], scales={'x':dt_x, 'y':sc_y})
    lineplot = Figure(marks = [line_marks])

    def state_selected_callback(selected_1, selected_2):
        if selected_2['data'] != None:
            fips_state=fips[fips['fips']==int(selected_2['data']['id'])]
            lineplot.title='The {0} of {1}'.format(field, selected_2['data']['name'])
            line_marks.x, line_marks.y = fips_state['year'], fips_state[field]
            ax_x.label, ax_y.label ='Year', field
            lineplot.axes = [ax_x, ax_y]
            line_marks.visible = True
        else:#when click on the background, lineplot will not be shown
            ax_y.label, ax_x.lable, lineplot.title = '', '', ''
            lineplot.axes = []
            line_marks.visible = False
      
    map_res.on_element_click(state_selected_callback)
    map_res.on_background_click(state_selected_callback)
    #clicking background will hidden the 
    
    return HBox([map_fig, lineplot])
    

A Jupyter Widget